In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

In [3]:
#df = pd.read_excel("IWA.xlsx", sheet_name='Final Raw Sample(3%)')
df = pd.read_excel("IWA.xlsx", sheet_name='Final Raw Sample(0%)')
df["Revenue"] = df["Total Environmental Cost"]/df["Environmental Intensity (Sales)"]
df["Operating Income"] = df["Total Environmental Cost"]/df["Environmental Intensity (Op Inc)"]
df["Environmental Intensity (Op Inc)"] = df["Environmental Intensity (Op Inc)"]*100
df["Environmental Intensity (Sales)"] = df["Environmental Intensity (Sales)"]*100
df.head()

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed,Revenue,Operating Income
0,2019,SAGA PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Multi-line Insurance,Activities auxiliary to financial intermediati...,-2.887178,-13.025357,-3.184231e+07,-3.115075e+07,-7184.203318,...,-4.739468,-1.027193,-3584.970569,-5.649112,70.667599,70.667599,-1297.277948,0.006135,1.102887e+09,244464000.0
1,2019,BURSA MALAYSIA BHD,MALAYSIA,Financial Exchanges & Data,Activities auxiliary to financial intermediati...,-1.677157,-3.465639,-1.968379e+06,-1.924910e+06,-451.342112,...,-1.410813,-1.207108,-222.196310,-1.681590,10.138780,10.138780,-79.398691,0.043215,1.173640e+08,56797000.0
2,2019,INTERTEK GROUP PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Research & Consulting Services,Activities auxiliary to financial intermediati...,-1.529690,-9.487849,-6.059927e+07,-5.928166e+07,-13774.014902,...,-17.024036,-3.689647,-6861.392776,-20.291452,253.836024,253.836024,-2470.054721,0.011467,3.961539e+09,638704000.0
3,2019,JSE LIMITED,SOUTH AFRICA,Financial Exchanges & Data,Activities auxiliary to financial intermediati...,-1.462497,NaN,-2.290124e+06,-2.239814e+06,-510.210093,...,-0.189720,-1.009642,-253.366805,-0.226133,-3.169102,-3.169102,-92.619013,0.016390,1.565900e+08,NaN
4,2019,BUREAU VERITAS SA,FRANCE,Research & Consulting Services,Activities auxiliary to financial intermediati...,-0.699273,-5.095678,-3.997865e+07,-3.910761e+07,-9330.459280,...,-37.818819,-9.136488,-4606.916825,-45.077368,586.030400,586.030400,-1632.997165,0.033005,5.717172e+09,784560000.0


In [1]:
harvard_set = ["Working Capacity", "Fish Production Capacity", "Crop Production Capacity",
                      "Meat Production Capacity", "Biodiversity", "Abiotic Resources",
                      "Water production capacity (Drinking water & Irrigation Water)", "Wood Production Capacity"]

sdg_set = ["SDG 1.5", "SDG 2.1", "SDG 2.2", "SDG 2.3", "SDG 2.4", "SDG 3.3", 
                         "SDG 3.4", "SDG 3.9", "SDG 6", "SDG 12.2", "SDG 14.1", "SDG 14.2", 
                         "SDG 14.3", "SDG 14.c", "SDG 15.1", "SDG 15.2", "SDG 15.5"]

independentVariables = sdg_set
dependentVariables = "Environmental Intensity (Sales)"

In [38]:

# Define your dependent variable
df[dependentVariables] = pd.to_numeric(df[dependentVariables], errors='coerce')
y = df[dependentVariables]

# Define your independent variables (including Year dummies)
X = df[independentVariables]

# Add Year dummies to the independent variables
year_dummies = pd.get_dummies(df['Year'], drop_first=True)
year_dummies = year_dummies.astype(int)
X = pd.concat([X, year_dummies], axis=1)

# Add a constant (intercept) to the independent variables
X = sm.add_constant(X)
model = sm.OLS(y, X.dropna()).fit()  
adjusted_r_squared = 1 - (1 - model.rsquared) * (len(y) - 1) / (len(y) - X.shape[1] - 1)

print(f'Adjusted R-squared (Fixed on Year): {adjusted_r_squared:.4f}')

Adjusted R-squared (Fixed on Year): 0.3028


In [39]:
# Create dummy variables for fixed effects on Year and GICS Sub-Industry
year_dummies = pd.get_dummies(df['Year'], drop_first=True)
year_dummies = year_dummies.astype(int)
subindustry_dummies = pd.get_dummies(df['GICS Sub-Industry'], drop_first=True)
subindustry_dummies = subindustry_dummies.astype(int)

# Define your dependent variable
y = df['Environmental Intensity (Sales)']

# Define your independent variables (including Year and Sub-Industry dummies)
X = df[['Total Environmental Cost', 'Working Capacity', 'Fish Production Capacity', 'Crop Production Capacity', 'Meat Production Capacity', 'Biodiversity', 'Abiotic Resources', 'Water production capacity (Drinking water & Irrigation Water)', 'Wood Production Capacity', 'SDG 1.5', 'SDG 2.1', 'SDG 2.2', 'SDG 2.3', 'SDG 2.4', 'SDG 3.3', 'SDG 3.4', 'SDG 3.9', 'SDG 6', 'SDG 12.2', 'SDG 14.1', 'SDG 14.2', 'SDG 14.3', 'SDG 14.c', 'SDG 15.1', 'SDG 15.2', 'SDG 15.5']]

# Add Year and Sub-Industry dummies to the independent variables
X = pd.concat([X, year_dummies, subindustry_dummies], axis=1)

# Add a constant (intercept) to the independent variables
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Calculate the adjusted R-squared
adjusted_r_squared = 1 - (1 - model.rsquared) * (len(y) - 1) / (len(y) - X.shape[1] - 1)

print(f'Adjusted R-squared (Fixed on Year and GICS Sub-Industry): {adjusted_r_squared:.4f}')


Adjusted R-squared (Fixed on Year and GICS Sub-Industry): 0.5621


In [40]:
# Create dummy variables for fixed effects on Year and GICS Sub-Industry
year_dummies = pd.get_dummies(df['Year'], drop_first=True)
year_dummies = year_dummies.astype(int)
subindustry_dummies = pd.get_dummies(df['Industry (Exiobase)'], drop_first=True)
subindustry_dummies = subindustry_dummies.astype(int)

# Define your dependent variable
y = df['Environmental Intensity (Sales)']

# Define your independent variables (including Year and Sub-Industry dummies)
X = df[['Total Environmental Cost', 'Working Capacity', 'Fish Production Capacity', 'Crop Production Capacity', 'Meat Production Capacity', 'Biodiversity', 'Abiotic Resources', 'Water production capacity (Drinking water & Irrigation Water)', 'Wood Production Capacity', 'SDG 1.5', 'SDG 2.1', 'SDG 2.2', 'SDG 2.3', 'SDG 2.4', 'SDG 3.3', 'SDG 3.4', 'SDG 3.9', 'SDG 6', 'SDG 12.2', 'SDG 14.1', 'SDG 14.2', 'SDG 14.3', 'SDG 14.c', 'SDG 15.1', 'SDG 15.2', 'SDG 15.5']]

# Add Year and Sub-Industry dummies to the independent variables
X = pd.concat([X, year_dummies, subindustry_dummies], axis=1)

# Add a constant (intercept) to the independent variables
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Calculate the adjusted R-squared
adjusted_r_squared = 1 - (1 - model.rsquared) * (len(y) - 1) / (len(y) - X.shape[1] - 1)

print(f'Adjusted R-squared (Fixed on Year and Industry (Exiobase)): {adjusted_r_squared:.4f}')


Adjusted R-squared (Fixed on Year and Industry (Exiobase)): 0.5291
